In [1]:
import pandas as pd 

In [2]:
df = pd.read_csv('global_energy_annual.csv', keep_default_na=False, na_values=[''])

In [3]:
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,1900,1901,1902,1903,1904,1905,1906,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,IS,Iceland,total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,18.550000,19.239000,19.830000,19.487000,18.778694,19.226948,19.697483,19.849665,19.082785,18.810204
1,IS,Iceland,primary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,61.371881,64.628095,70.854688,69.891429,57.441930,81.766586,85.260083,86.418968,86.071569,NaN
2,IS,Iceland,primary-coal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.126619,1.131639,1.309183,1.235183,0.961232,1.361305,1.346902,0.968476,1.252253,NaN
3,IS,Iceland,primary-oil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.646074,6.265873,6.498406,6.470477,7.309943,8.265474,10.354054,10.943816,11.348905,NaN
4,IS,Iceland,primary-renewables,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,53.599188,57.230587,63.047098,62.185769,61.463145,NaN,NaN,NaN,NaN,NaN


### Create Year Column

In [4]:
year_cols = [c for c in df.columns if c.isdigit()]

df_long = df.melt(
    id_vars=[c for c in df.columns if c not in year_cols],
    value_vars=year_cols,
    var_name='Year',
    value_name='Energy (TWh)'
)

df_long['Year'] = df_long['Year'].astype(int)

In [5]:
df_long.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Year,Energy (TWh)
0,IS,Iceland,total,1900,NaN
1,IS,Iceland,primary,1900,NaN
2,IS,Iceland,primary-coal,1900,NaN
3,IS,Iceland,primary-oil,1900,NaN
4,IS,Iceland,primary-renewables,1900,NaN


In [6]:
df_long.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 618660 entries, 0 to 618659
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    618660 non-null  object 
 1   Unnamed: 1    618660 non-null  object 
 2   Unnamed: 2    618660 non-null  object 
 3   Year          618660 non-null  int64  
 4   Energy (TWh)  170521 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 23.6+ MB


### Renaming Columns

In [7]:
df_long = df_long.rename(columns={
    'Unnamed: 0': 'ISO',
    'Unnamed: 1': 'Country',
    'Unnamed: 2': 'Energy Category'
})

In [8]:
df_long.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 618660 entries, 0 to 618659
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   ISO              618660 non-null  object 
 1   Country          618660 non-null  object 
 2   Energy Category  618660 non-null  object 
 3   Year             618660 non-null  int64  
 4   Energy (TWh)     170521 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 23.6+ MB


### Cleaning 'ISO' and 'Country'

In [9]:
df_iso = df_long.copy()

In [10]:
df_iso['ISO'].isna().sum()

np.int64(0)

In [11]:
df_iso['Country'].isna().sum()

np.int64(0)

In [12]:
df_iso['Country'].unique()

array(['Iceland', 'Norway', 'Switzerland', 'France', 'Sweden', 'Finland',
       'Brazil', 'Denmark', 'Slovakia', 'New Zealand',
       'Special region: US-SD', 'Canada', 'Special region: US-WA',
       'Spain', 'Slovenia', 'Austria', 'Colombia', 'EU', 'Chile',
       'Portugal', 'Special region: US-NH', 'Bulgaria',
       'Special region: US-IL', 'Special region: US-KS',
       'Special region: US-IA', 'Belgium', 'Special region: US-OR',
       'Special region: US-ME', 'Czechia', 'Romania', 'Germany',
       'Special region: US-SC', 'United Kingdom', 'Latvia',
       'Special region: US-MT', 'Lithuania', 'Hungary',
       'Special region: US-NM', 'Special region: US-CA', 'Croatia',
       'Netherlands', 'Greece', 'Special region: US-AZ',
       'Special region: US-NE', 'Special region: US-ID',
       'Special region: US-NV', 'Argentina', 'Special region: US-MN',
       'Turkey', 'Special region: US-NY', 'Special region: US-OK',
       'South Korea', 'United States', 'Ireland', 'Specia

In [13]:
aggregates = [
    'The World', 'EU', 'W2',
    'Sub-Saharan Africa'
]

#Remove Regions that are not Individual Countries/Territories
df_iso = df_iso[
    ~df_iso['Country'].isin(aggregates)
] 

# Remove US States
df_iso = df_iso[
    ~df_iso['Country'].str.startswith('Special region: US')
]

In [14]:
df_iso.info()

<class 'pandas.core.frame.DataFrame'>
Index: 473130 entries, 0 to 618659
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   ISO              473130 non-null  object 
 1   Country          473130 non-null  object 
 2   Energy Category  473130 non-null  object 
 3   Year             473130 non-null  int64  
 4   Energy (TWh)     145561 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 21.7+ MB


###  Cleaning 'Energy (TWh)'

In [15]:
df_energy = df_iso.copy()

In [16]:
df_energy = df_energy.dropna(subset=['Energy (TWh)'])

In [17]:
df_energy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 145561 entries, 348610 to 616191
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   ISO              145561 non-null  object 
 1   Country          145561 non-null  object 
 2   Energy Category  145561 non-null  object 
 3   Year             145561 non-null  int64  
 4   Energy (TWh)     145561 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 6.7+ MB


### Save Cleaned Dataset

In [18]:
df_clean = df_energy.copy().reset_index(drop=True)

In [19]:
df_clean.head()

,ISO,Country,Energy Category,Year,Energy (TWh)
0,IS,Iceland,total,1971,1.621000
1,IS,Iceland,primary,1971,10.494829
2,IS,Iceland,primary-coal,1971,0.007327
3,IS,Iceland,primary-oil,1971,5.568222
4,IS,Iceland,primary-renewables,1971,4.919278


In [20]:
df_clean.tail()

,ISO,Country,Energy Category,Year,Energy (TWh)
145556,MD,Moldova,biofuels,2025,0.013716
145557,MD,Moldova,coal,2025,0.000000
145558,MD,Moldova,gas,2025,1.722018
145559,MD,Moldova,oil,2025,0.000000
145560,MD,Moldova,trade,2025,2.471535


In [21]:
print(f"Total rows: {len(df_clean):,}")
print(f"Date range: {df_clean['Year'].min()} to {df_clean['Year'].max()}")
print(f"Countries: {df_clean['Country'].nunique()}")
print(f"Energy types: {df_clean['Energy Category'].nunique()}")
print(f"Missing values:\n{df_clean.isnull().sum()}")

Total rows: 145,561
Date range: 1971 to 2025
Countries: 211
Energy types: 36
Missing values:
ISO                0
Country            0
Energy Category    0
Year               0
Energy (TWh)       0
dtype: int64


In [22]:
df_clean.to_csv('global_energy_data.csv', index=False)